In [1]:
# FASE 4_PROYECTO ZTNA_202016907_Proyecto de Grado_ANDRES_YORMAN_AIDA

# CASO DE USO #1: Automatización de respuesta a incidentes con Cortex XSOAR

# Requisitos: pip install requests pandas matplotlib
import requests
import json
import pandas as pd
import time
from datetime import datetime
import os
from google.colab import files
import matplotlib.pyplot as plt

# Configuración de las credenciales
# En un entorno de producción, se recomienda usar variables de entorno o un sistema de gestión de secretos
API_KEY = "your_xsoar_api_key"  # Se utiliza la clave API
BASE_URL = "https://xsoar.ejemplo.com"  # URL base de la instancia de Cortex XSOAR

# Headers para las solicitudes API
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': API_KEY
}

# Función para obtener los incidentes activos con alta severidad
def get_high_severity_incidents():
    url = f"{BASE_URL}/incidents/search"

    # Búsqueda de incidentes de severidad alta/crítica relacionados con endpoints
    query = {
        "filter": {
            "query": "severity:high OR severity:critical",
            "period": {
                "by": "day",
                "fromValue": 1
            }
        }
    }

    try:
        response = requests.post(url, headers=headers, json=query)
        response.raise_for_status()  # Lanzar excepción si la respuesta no es 2xx

        incidents = response.json().get('data', [])
        print(f"Se encontraron {len(incidents)} incidentes de alta severidad")

        return incidents
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener incidentes: {e}")
        return []

# Función para extraer detalles relevantes de los incidentes
def extract_incident_details(incidents):
    details = []

    for incident in incidents:
        # Extraer detalles del endpoint afectado (si está disponible)
        endpoint = incident.get('CustomFields', {}).get('hostname', 'Unknown')
        ip_address = incident.get('CustomFields', {}).get('srcip', 'Unknown')

        details.append({
            'incident_id': incident.get('id'),
            'name': incident.get('name'),
            'severity': incident.get('severity'),
            'status': incident.get('status'),
            'created': incident.get('created'),
            'endpoint': endpoint,
            'ip_address': ip_address
        })

    return details

# Función para iniciar un playbook de cuarentena
def initiate_quarantine_playbook(incident_id, hostname, ip_address):
    url = f"{BASE_URL}/incident/execute/command"

    # Datos necesarios para ejecutar el playbook de cuarentena
    payload = {
        "incident_id": incident_id,
        "command": "cortex-xdr-isolate-endpoint",
        "arguments": {
            "endpoint_id": hostname,
            "ip": ip_address,
            "isolation_status": "containment"
        }
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()

        print(f"Playbook de cuarentena iniciado para el incidente {incident_id}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error al iniciar playbook: {e}")
        return False

# Función principal
def main():
    print("Iniciando automatización de respuesta a incidentes...")

    # 1. Obtener incidentes de alta severidad
    incidents = get_high_severity_incidents()

    if not incidents:
        print("No se encontraron incidentes que requieran atención inmediata.")
        return

    # 2. Extraer detalles relevantes
    incident_details = extract_incident_details(incidents)

    # 3. Convertir a DataFrame para mejor visualización
    df = pd.DataFrame(incident_details)
    print("\nResumen de incidentes de alta severidad:")
    print(df)

    # 4. Identificar endpoints que requieren cuarentena
    # Se ponen en cuarentena todos los endpoints de incidentes críticos
    quarantine_candidates = df[df['severity'] == 'critical']

    if quarantine_candidates.empty:
        print("\nNo hay endpoints que requieran cuarentena inmediata.")
        return

    print(f"\nSe identificaron {len(quarantine_candidates)} endpoints para cuarentena:")
    print(quarantine_candidates[['incident_id', 'endpoint', 'ip_address']])

    # 5. Iniciar playbooks de cuarentena
    quarantine_results = []
    for _, row in quarantine_candidates.iterrows():
        result = initiate_quarantine_playbook(
            row['incident_id'],
            row['endpoint'],
            row['ip_address']
        )
        quarantine_results.append({
            'incident_id': row['incident_id'],
            'endpoint': row['endpoint'],
            'result': 'Success' if result else 'Failed'
        })

    # 6. Mostrar resultados de las acciones de cuarentena
    results_df = pd.DataFrame(quarantine_results)
    print("\nResultados de las acciones de cuarentena:")
    print(results_df)

    # 7. Generar visualización
    success_count = (results_df['result'] == 'Success').sum()
    failed_count = (results_df['result'] == 'Failed').sum()

    plt.figure(figsize=(10, 6))
    plt.bar(['Éxito', 'Fallido'], [success_count, failed_count], color=['green', 'red'])
    plt.title('Resultados de Acciones de Cuarentena')
    plt.ylabel('Número de Endpoints')
    plt.savefig('quarantine_results.png')
    plt.close()

    # 8. Exportar resultados para análisis posterior
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_df.to_csv(f'quarantine_actions_{timestamp}.csv', index=False)

    # Como se creo en Colab, descargamos los archivos generados
    try:
        from google.colab import files
        files.download(f'quarantine_actions_{timestamp}.csv')
        files.download('quarantine_results.png')
    except:
        print(f"Los resultados se han guardado en 'quarantine_actions_{timestamp}.csv'")
        print("La visualización se ha guardado en 'quarantine_results.png'")

# Punto de entrada principal
if __name__ == "__main__":
    main()

Iniciando automatización de respuesta a incidentes...
Error al obtener incidentes: HTTPSConnectionPool(host='xsoar.ejemplo.com', port=443): Max retries exceeded with url: /incidents/search (Caused by SSLError(SSLError(1, '[SSL: TLSV1_UNRECOGNIZED_NAME] tlsv1 unrecognized name (_ssl.c:1016)')))
No se encontraron incidentes que requieran atención inmediata.
